# Two-Tower Recommendation System for Coupon Recommendation

This notebook implements a Two-Tower recommendation system using TensorFlow Recommenders for coupon recommendation analysis.

## Dataset Overview
- **Transaction Data**: Customer purchase history
- **Coupon Data**: Available coupons and their details
- **Household Demographics**: Customer demographic information
- **Product Data**: Product information and categories

## Model Architecture
- **User Tower**: Encodes user/household information
- **Item Tower**: Encodes coupon/product information
- **Retrieval Task**: Learns to match users with relevant coupons


In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print("TensorFlow version:", tf.__version__)
print("TensorFlow Recommenders version:", tfrs.__version__)
print("Pandas version:", pd.__version__)
print("NumPy version:", np.__version__)


In [ ]:
# Load the datasets
print("Loading datasets...")

# Load transaction data
transaction_data = pd.read_csv('Dataset/transaction_data.csv')
print(f"Transaction data shape: {transaction_data.shape}")

# Load coupon data
coupon = pd.read_csv('Dataset/coupon.csv')
print(f"Coupon data shape: {coupon.shape}")

# Load household demographics
hh_demographic = pd.read_csv('Dataset/hh_demographic.csv')
print(f"Household demographic data shape: {hh_demographic.shape}")

# Load product data
product = pd.read_csv('Dataset/product.csv')
print(f"Product data shape: {product.shape}")

# Load coupon redemption data
coupon_redempt = pd.read_csv('Dataset/coupon_redempt.csv')
print(f"Coupon redemption data shape: {coupon_redempt.shape}")

print("\nDataset loading completed!")


In [ ]:
# Data exploration and preprocessing
print("=== Data Exploration ===")

# Transaction data exploration
print("\nTransaction Data Info:")
print(transaction_data.info())
print("\nTransaction Data Head:")
print(transaction_data.head())

# Coupon data exploration
print("\nCoupon Data Info:")
print(coupon.info())
print("\nCoupon Data Head:")
print(coupon.head())

# Household demographic exploration
print("\nHousehold Demographic Info:")
print(hh_demographic.info())
print("\nHousehold Demographic Head:")
print(hh_demographic.head())

# Product data exploration
print("\nProduct Data Info:")
print(product.info())
print("\nProduct Data Head:")
print(product.head())


In [ ]:
# Data preprocessing and feature engineering
print("=== Data Preprocessing ===")

# Create user-item interaction data
# We'll use household_key as user_id and COUPON_UPC as item_id
print("Creating user-item interaction data...")

# Get unique households and coupons
unique_households = transaction_data['household_key'].unique()
unique_coupons = coupon['COUPON_UPC'].dropna().unique()

print(f"Number of unique households: {len(unique_households)}")
print(f"Number of unique coupons: {len(unique_coupons)}")

# Create interaction data from coupon redemptions
interaction_data = coupon_redempt[['household_key', 'COUPON_UPC']].dropna()
interaction_data = interaction_data.rename(columns={'household_key': 'user_id', 'COUPON_UPC': 'coupon_id'})

print(f"Interaction data shape: {interaction_data.shape}")
print("Interaction data head:")
print(interaction_data.head())

# Check for missing values
print(f"\nMissing values in interaction data:")
print(interaction_data.isnull().sum())


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs
import warnings

warnings.filterwarnings('ignore')

# Set the path to Dataset folder
path = "Dataset/"

In [ ]:
# Read all CSV files
campaign_desc = pd.read_csv(path + "campaign_desc.csv")
campaign_table = pd.read_csv(path + "campaign_table.csv")
coupon_redempt = pd.read_csv(path + "coupon_redempt.csv") ## 1 counpon_unc can have multiple product_id: 556 nunique from 2318
coupon = pd.read_csv(path + "coupon.csv")
hh_demographic = pd.read_csv(path + "hh_demographic.csv")
product = pd.read_csv(path + "product.csv")
transaction_data = pd.read_csv(path + "transaction_data.csv")

# Check the dataframes
print("DataFrames loaded successfully!")
print(f"campaign_desc shape: {campaign_desc.shape}")
print(f"campaign_table shape: {campaign_table.shape}")
print(f"coupon_redempt shape: {coupon_redempt.shape}")
print(f"coupon shape: {coupon.shape}")
print(f"hh_demographic shape: {hh_demographic.shape}")
print(f"product shape: {product.shape}")
print(f"transaction_data shape: {transaction_data.shape}")



DataFrames loaded successfully!
campaign_desc shape: (30, 4)
campaign_table shape: (7208, 3)
coupon_redempt shape: (2318, 4)
coupon shape: (124548, 3)
hh_demographic shape: (801, 8)
product shape: (92353, 7)
transaction_data shape: (2595732, 12)


In [ ]:
# Show headers for each dataset
print("Headers for each dataset:")
print("=" * 50)

datasets = {
    'campaign_desc': campaign_desc,
    'campaign_table': campaign_table,
    'coupon_redempt': coupon_redempt,
    'coupon': coupon,
    'hh_demographic': hh_demographic,
    'product': product,
    'transaction_data': transaction_data
}

for name, df in datasets.items():
    print(f"\n{name}:")
    print(f"Columns: {list(df.columns)}")


Headers for each dataset:

campaign_desc:
Columns: ['DESCRIPTION', 'CAMPAIGN', 'START_DAY', 'END_DAY']

campaign_table:
Columns: ['DESCRIPTION', 'household_key', 'CAMPAIGN']

coupon_redempt:
Columns: ['household_key', 'DAY', 'COUPON_UPC', 'CAMPAIGN']

coupon:
Columns: ['COUPON_UPC', 'PRODUCT_ID', 'CAMPAIGN']

hh_demographic:
Columns: ['AGE_DESC', 'MARITAL_STATUS_CODE', 'INCOME_DESC', 'HOMEOWNER_DESC', 'HH_COMP_DESC', 'HOUSEHOLD_SIZE_DESC', 'KID_CATEGORY_DESC', 'household_key']

product:
Columns: ['PRODUCT_ID', 'MANUFACTURER', 'DEPARTMENT', 'BRAND', 'COMMODITY_DESC', 'SUB_COMMODITY_DESC', 'CURR_SIZE_OF_PRODUCT']

transaction_data:
Columns: ['household_key', 'BASKET_ID', 'DAY', 'PRODUCT_ID', 'QUANTITY', 'SALES_VALUE', 'STORE_ID', 'RETAIL_DISC', 'TRANS_TIME', 'WEEK_NO', 'COUPON_DISC', 'COUPON_MATCH_DISC']


In [ ]:
hh_demographic.head(10)
hh_demographic.drop(["AGE_DESC",
                     "MARITAL_STATUS_CODE",
                     "INCOME_DESC",
                     "HH_COMP_DESC",
                     "HOUSEHOLD_SIZE_DESC",
                     "KID_CATEGORY_DESC",],
                    axis=1,inplace = True)

In [ ]:
campaign_desc.head()


,DESCRIPTION,CAMPAIGN,START_DAY,END_DAY
0,TypeB,24,659,719
1,TypeC,15,547,708
2,TypeB,25,659,691
3,TypeC,20,615,685
4,TypeB,23,646,684


In [ ]:
# Create timestamp feature from DAY and TRANS_TIME
def create_timestamp(day, trans_time):
    """Convert DAY and TRANS_TIME to datetime timestamp"""
    # DAY appears to be day number from some reference point
    # TRANS_TIME is in HHMM format (e.g., 1631 = 16:31)
    base_date = datetime(2010, 3, 24)  # Arbitrary base date
    date_part = base_date + timedelta(days=int(day) - 1)
    
    # Convert TRANS_TIME to time
    hours = int(trans_time) // 100
    minutes = int(trans_time) % 100
    time_part = timedelta(hours=hours, minutes=minutes)
    
    return date_part + time_part

# Add timestamp to transaction_data
transaction_data['timestamp'] = transaction_data.apply(
    lambda row: create_timestamp(row['DAY'], row['TRANS_TIME']), axis=1
)


In [ ]:
transaction_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2595732 entries, 0 to 2595731
Data columns (total 13 columns):
 #   Column             Dtype         
---  ------             -----         
 0   household_key      int64         
 1   BASKET_ID          int64         
 2   DAY                int64         
 3   PRODUCT_ID         int64         
 4   QUANTITY           int64         
 5   SALES_VALUE        float64       
 6   STORE_ID           int64         
 7   RETAIL_DISC        float64       
 8   TRANS_TIME         int64         
 9   WEEK_NO            int64         
 10  COUPON_DISC        float64       
 11  COUPON_MATCH_DISC  float64       
 12  timestamp          datetime64[ns]
dtypes: datetime64[ns](1), float64(4), int64(8)
memory usage: 257.5 MB


In [ ]:
coupon_redempt = coupon_redempt.merge(
    coupon[['COUPON_UPC', 'PRODUCT_ID']], 
    on='COUPON_UPC', 
    how='left'
)

In [ ]:
coupon_redempt.head()

,household_key,DAY,COUPON_UPC,CAMPAIGN,PRODUCT_ID
0,1,421,10000085364,8,100512
1,1,421,10000085364,8,527731
2,1,421,10000085364,8,1054539
3,1,421,10000085364,8,802268
4,1,421,10000085364,8,846907


In [ ]:
transaction_with_coupons = transaction_data.merge(
    coupon_redempt,
    on=['household_key', 'DAY', 'PRODUCT_ID'],
    how='left'
)

In [ ]:
transaction_with_coupons

,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC,timestamp,COUPON_UPC,CAMPAIGN
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0,2010-03-24 16:31:00,NaN,NaN
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0,2010-03-24 16:31:00,NaN,NaN
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0,2010-03-24 16:31:00,NaN,NaN
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0,2010-03-24 16:31:00,NaN,NaN
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0,2010-03-24 16:31:00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2598065,1598,42305362535,711,92130,1,0.99,3228,0.00,1520,102,0.0,0.0,2012-03-03 15:20:00,NaN,NaN
2598066,1598,42305362535,711,114102,1,8.89,3228,0.00,1520,102,0.0,0.0,2012-03-03 15:20:00,NaN,NaN
2598067,1598,42305362535,711,133449,1,6.99,3228,0.00,1520,102,0.0,0.0,2012-03-03 15:20:00,NaN,NaN
2598068,1598,42305362535,711,6923644,1,4.50,3228,-0.49,1520,102,0.0,0.0,2012-03-03 15:20:00,NaN,NaN


In [ ]:
transaction_with_coupons.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2598070 entries, 0 to 2598069
Data columns (total 15 columns):
 #   Column             Dtype         
---  ------             -----         
 0   household_key      int64         
 1   BASKET_ID          int64         
 2   DAY                int64         
 3   PRODUCT_ID         int64         
 4   QUANTITY           int64         
 5   SALES_VALUE        float64       
 6   STORE_ID           int64         
 7   RETAIL_DISC        float64       
 8   TRANS_TIME         int64         
 9   WEEK_NO            int64         
 10  COUPON_DISC        float64       
 11  COUPON_MATCH_DISC  float64       
 12  timestamp          datetime64[ns]
 13  COUPON_UPC         float64       
 14  CAMPAIGN           float64       
dtypes: datetime64[ns](1), float64(6), int64(8)
memory usage: 297.3 MB


In [ ]:
transactions_with_coupons_sum = transaction_with_coupons['COUPON_UPC'].notna().sum()
print(f"Transactions with coupon information: {transactions_with_coupons_sum:,}")
print(f"Percentage of transactions with coupons: {transactions_with_coupons_sum/len(transaction_with_coupons)*100:.2f}%")


Transactions with coupon information: 6,607
Percentage of transactions with coupons: 0.25%


In [ ]:
transaction_with_coupons["SALES_VALUE"] = transaction_with_coupons["SALES_VALUE"] - transaction_with_coupons["RETAIL_DISC"]
transaction_with_coupons.drop("RETAIL_DISC", axis=1, inplace=True)
transaction_with_coupons["Use_Coupon"] = transaction_with_coupons["COUPON_UPC"].notna().astype(int)
transaction_with_coupons

,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,TRANS_TIME,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC,timestamp,COUPON_UPC,CAMPAIGN,Use_Coupon
0,2375,26984851472,1,1004906,1,1.99,364,1631,1,0.0,0.0,2010-03-24 16:31:00,NaN,NaN,0
1,2375,26984851472,1,1033142,1,0.82,364,1631,1,0.0,0.0,2010-03-24 16:31:00,NaN,NaN,0
2,2375,26984851472,1,1036325,1,1.29,364,1631,1,0.0,0.0,2010-03-24 16:31:00,NaN,NaN,0
3,2375,26984851472,1,1082185,1,1.21,364,1631,1,0.0,0.0,2010-03-24 16:31:00,NaN,NaN,0
4,2375,26984851472,1,8160430,1,1.89,364,1631,1,0.0,0.0,2010-03-24 16:31:00,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2598065,1598,42305362535,711,92130,1,0.99,3228,1520,102,0.0,0.0,2012-03-03 15:20:00,NaN,NaN,0
2598066,1598,42305362535,711,114102,1,8.89,3228,1520,102,0.0,0.0,2012-03-03 15:20:00,NaN,NaN,0
2598067,1598,42305362535,711,133449,1,6.99,3228,1520,102,0.0,0.0,2012-03-03 15:20:00,NaN,NaN,0
2598068,1598,42305362535,711,6923644,1,4.99,3228,1520,102,0.0,0.0,2012-03-03 15:20:00,NaN,NaN,0


In [ ]:
# Fix for StringLookup layer - convert household_key to strings
# First, let's get the unique household keys and convert them to strings
unique_household_key = list(transaction_data['household_key'].unique().astype(str))
unique_coupon_upc = list(coupon["COUPON_UPC"].dropna().unique().astype(str))

embedding_dimension = 32

context_model = tf.keras.Sequential([
    tf.keras.Input(shape=(), dtype=tf.string),
    tf.keras.layers.StringLookup(vocabulary=unique_household_key,
                                 mask_token=None, num_oov_indices=1),
    tf.keras.layers.Embedding(len(unique_household_key)+1, embedding_dimension),
])

coupon_model = tf.keras.Sequential([
    tf.keras.Input(shape=(), dtype=tf.string),
    tf.keras.layers.StringLookup(vocabulary=unique_household_key,
                                 mask_token=None, num_oov_indices=1),
    tf.keras.layers.Embedding(len(unique_household_key)+1, embedding_dimension),
])


In [ ]:
# dataset ของ coupon id เป็น string
candidates_ids_ds = tf.data.Dataset.from_tensor_slices(unique_coupon_upc)

# สำคัญ: batch ก่อน แล้วค่อย map โมเดล
candidates_emb_ds = candidates_ids_ds.batch(128).map(
    coupon_model, num_parallel_calls=tf.data.AUTOTUNE
)

metrics = tfrs.metrics.FactorizedTopK(
    candidates=candidates_emb_ds,
    ks=[1,5,10,20,50,100]
)
task = tfrs.tasks.Retrieval(metrics=metrics)


ValueError: Cannot convert '('c', 'o', 'u', 'n', 't', 'e', 'r')' to a shape. Found invalid entry 'c' of type '<class 'str'>'. 

In [ ]:

train,test = train_test_split(
    transaction_with_coupons,
    test_size=0.2,
    random_state=42,
    stratify=transaction_with_coupons["Use_Coupon"])

def df_to_ds(df):
    return tf.data.Dataset.from_tensor_slices(
        {
            "user_id": df["household_key"].astype(str),
            "coupon_id": df["COUPON_UPC"].astype(str)
        }
    )
    
train_ds = df_to_ds(train)
test_ds = df_to_ds(test)
